In [1]:
# biblioteki
from src.dataclass import TextDataSet, ImageDataSet
from transformers import ViTImageProcessor, AutoModelForImageClassification
import torch
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import  DataLoader
from torchvision.transforms import transforms, ToPILImage, ToTensor

In [2]:
# ładowanie danych
image_data_set = ImageDataSet('../datasets/train_set')
ImageDataSet.use_rgb(image_data_set)

dataloader = DataLoader(
    image_data_set,
    batch_size=8,
    shuffle=True,
    num_workers=8,
)

In [3]:
# model z hugingface
extractor = ViTImageProcessor.from_pretrained("DunnBC22/dit-base-Business_Documents_Classified_v2")

model = AutoModelForImageClassification.from_pretrained("DunnBC22/dit-base-Business_Documents_Classified_v2")

In [4]:
# zmiana głowicy
model.classifier = torch.nn.Linear(in_features=model.classifier.in_features, out_features=21)

In [5]:
# zamrożenie parametrów

for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

In [19]:
train_data = [torch.tensor(x[0]) for x in tqdm(image_data_set)]

labels = [torch.zeros(21)[x[1]] for x in tqdm(image_data_set)]

100%|██████████| 10849/10849 [01:58<00:00, 91.87it/s]


In [18]:
# trening głowicy
criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

num_epochs = 10

losses = []



for epoch in tqdm(range(num_epochs)):
    running_loss = 0
    for data in tqdm(image_data_set):
        image, label = data
        optimizer.zero_grad()


        inputs = extractor(images=image, return_tensors="pt")
        outputs = model(**inputs).logits

        one_hot = torch.zeros(21)
        one_hot[label] = 1
        loss = criterion(outputs, one_hot.reshape(outputs.shape))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    al = running_loss / len(image_data_set)
    losses.append(al)

plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

  0%|          | 0/10 [03:25<?, ?it/s]


KeyboardInterrupt: 

In [7]:
for image, label in image_data_set:
    # Pextractor
    inputs = extractor(images=image, return_tensors="pt")

    # inferencja
    logits = model(**inputs).logits
    probs = torch.nn.functional.softmax(logits, dim=-1)

    # klasfikacja
    predicted_class = torch.argmax(probs).item()

    print("Przewidywana klasa:", predicted_class, "Prawdziwa klasa:", label)


Przewidywana klasa: 1 Prawdziwa klasa: 1
Przewidywana klasa: 1 Prawdziwa klasa: 1
Przewidywana klasa: 1 Prawdziwa klasa: 1
Przewidywana klasa: 0 Prawdziwa klasa: 1
Przewidywana klasa: 0 Prawdziwa klasa: 1
Przewidywana klasa: 1 Prawdziwa klasa: 1
Przewidywana klasa: 1 Prawdziwa klasa: 1
Przewidywana klasa: 0 Prawdziwa klasa: 1
Przewidywana klasa: 0 Prawdziwa klasa: 1
Przewidywana klasa: 1 Prawdziwa klasa: 1
Przewidywana klasa: 0 Prawdziwa klasa: 1
Przewidywana klasa: 1 Prawdziwa klasa: 1
Przewidywana klasa: 1 Prawdziwa klasa: 1
Przewidywana klasa: 0 Prawdziwa klasa: 1
Przewidywana klasa: 0 Prawdziwa klasa: 1


KeyboardInterrupt: 